# Exercice I

### Réaliser un correcteur d'orthographe en python pour la langue anglaise en se basant sur l'algorithme implémenté dans les TPs.

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import brown
from collections import Counter

In [2]:
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [3]:
vocab = list(brown.words())
vocab = [v.lower() for v in vocab]
vocab[:5]

['the', 'fulton', 'county', 'grand', 'jury']

Preprocessing

In [4]:
word_count_dict = Counter(vocab)
type(word_count_dict)

collections.Counter

In [5]:
def get_probs(word_count_dict):
    '''
    Input:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    Output:
        probs: A dictionary where keys are the words and the values are the probability that a word will occur. 
    '''
    probs = {}  # return this variable correctly
    
    ### START CODE HERE ###
    m = sum(word_count_dict.values())
    for key in word_count_dict.keys():
        probs[key] = word_count_dict[key] / m
    ### END CODE HERE ###
    return probs

In [6]:
probs = get_probs(word_count_dict)
type(probs)


dict

In [7]:
def generate_candidates_by_one_edit(word):
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [left + right[1:] for left, right in splits if right]
    switchs = [left + right[1] + right[0] + right[2:] for left, right in splits if len(right) > 1]
    replaces = [left + c + right[1:] for left, right in splits if right for c in alphabet]
    inserts = [left + c + right for left, right in splits for c in alphabet]

    return set(deletes + switchs + replaces + inserts)

In [8]:
cand = generate_candidates_by_one_edit('sould')
type(cand)

set

In [9]:
def get_candidate_by_two_edit(word):
    edit_two_set = set()
    edit_one = generate_candidates_by_one_edit(word)
    for dandidates in edit_one:
        edit_two = generate_candidates_by_one_edit(dandidates)
        edit_two_set.update(edit_two)
    return edit_two_set

In [10]:
edited_twice = get_candidate_by_two_edit('sould')
print(len(edited_twice))

36860


In [11]:
def get_candidate_by_three_edit(word):
  edit_three_set = set()
  edit_two = get_candidate_by_two_edit(word)
  for candidates in edit_two:
    edit_three = generate_candidates_by_one_edit(candidates)
    edit_three_set.update(edit_three)
  return edit_three_set

In [12]:
def get_valid_corrections(word,vocab):
  # candidates = get_candidate_by_two_edit(word)
  # valid_candidates=[]
  suggestions = list((word in vocab and word) or 
                     generate_candidates_by_one_edit(word).intersection(vocab) or 
                     get_candidate_by_two_edit(word).intersection(vocab) or 
                     get_candidate_by_three_edit(word).intersection(vocab))
  # for cand in candidates:
  #   if cand in vocab:
  #     valid_candidates.append(cand)
  # valid_candidates = [candidate for candidate in candidates if candidate in vocab]
  
  return suggestions

In [13]:
get_valid_corrections('sould',vocab)

['gould',
 'soul',
 'mould',
 'sold',
 'sound',
 'should',
 'could',
 'ould',
 'would',
 'shuld',
 'souls',
 'soule']

In [14]:
import re

In [15]:
def q1_solution(text,vocab):
  words = re.findall(r'\w+', text.lower())
  corrected_text=''
  for word in words:
    if word not in vocab :
      corrected_word = get_valid_corrections(word, vocab)
      if len(corrected_word) == 1:
        print("miss spelled word detected ("+word+") the suggested corrections is :"+ str(corrected_word))
      else:
        print("miss spelled word detected ("+word+") the suggested corrections are :"+ str(corrected_word))

#### Answer the question 1.1

In [16]:
q1_solution('I enjoy writinf texr edito',vocab)

miss spelled word detected (writinf) the suggested corrections is :['writing']
miss spelled word detected (texr) the suggested corrections are :['tex', 'text', 'tear']
miss spelled word detected (edito) the suggested corrections are :['edith', 'edit', 'editor']


### Implementer une variante du programme precedent qui utilise un modele de langue pour ordonner les corrections possibles par leurs probabilites selon le modele de langue et selon la distance minimale d'edition.


##### Min Edit Distance

In [17]:
def min_edit_distance(source, target, ins_cost=1, del_cost=1, rep_cost=2):
    '''
    Input:
        source: a string corresponding to the string you are starting with
        target: a string corresponding to the string you want to end with
        ins_cost: an integer setting the insert cost
        del_cost: an integer setting the delete cost
        rep_cost: an integer setting the replace cost
    Output:
        D: a matrix of len(source)+1 by len(target)+1 containing minimum edit distances
        med: the minimum edit distance (med) required to convert the source string to the target
    '''
    # use deletion and insert cost as  1
    m = len(source)
    n = len(target)
    # initialize cost matrix with zeros and dimensions (m+1,n+1)
    D = np.zeros((m + 1, n + 1), dtype=int)

    ### START CODE HERE (Replace instances of 'None' with your code) ###

    # Fill in column 0, from row 1 to row m, both inclusive
    for row in range(1, m + 1):  # Replace None with the proper range
        D[row, 0] = D[row - 1, 0] + del_cost

    # Fill in row 0, for all columns from 1 to n, both inclusive
    for col in range(1, n + 1):  # Replace None with the proper range
        D[0, col] = D[0, col - 1] + ins_cost

    # Loop through row 1 to row m, both inclusive
    for row in range(1, m + 1):

        # Loop through column 1 to column n, both inclusive
        for col in range(1, n + 1):

            # Intialize r_cost to the 'replace' cost that is passed into this function
            r_cost = rep_cost

            # Check to see if source character at the previous row
            # matches the target character at the previous column,
            if source[row - 1] == target[col - 1]:
                # Update the replacement cost to 0 if source and target are the same
                r_cost = 0

            # Update the cost at row, col based on previous entries in the cost matrix
            # Refer to the equation calculate for D[i,j] (the minimum of three calculated costs)
            D[row, col] = min([D[row - 1, col] + del_cost, D[row, col - 1] + ins_cost, D[row - 1, col - 1] + r_cost])

    # Set the minimum edit distance with the cost found at row m, column n
    med = D[m, n]

    ### END CODE HERE ###
    return med

In [18]:
def q2_solution_MED(text,vocab):
  words = re.findall(r'\w+', text.lower())
  corrected_text=''
  for word in words:
    if word not in vocab :
      corrected_word = get_valid_corrections(word, vocab)
      if len(corrected_word) == 1:
        print("miss spelled word detected ("+word+") the suggested corrections is :"+ str(corrected_word))
      else:
        print("miss spelled word detected ("+word+") the suggested corrections is : "+ str(corrected_word))
        sorted_corrections=sorted(corrected_word,key=lambda x:min_edit_distance(x,word))
        for sor_corr in sorted_corrections:
          print("miss spelled word detected ("+word+") the suggested corrections sorted by MED are :"+ str(sor_corr)+" min edit distance = "+str(min_edit_distance(sor_corr,word)))


In [19]:
q2_solution_MED('I enjoy writinf texr edito',vocab)

miss spelled word detected (writinf) the suggested corrections is :['writing']
miss spelled word detected (texr) the suggested corrections is : ['tex', 'text', 'tear']
miss spelled word detected (texr) the suggested corrections sorted by MED are :tex min edit distance = 1
miss spelled word detected (texr) the suggested corrections sorted by MED are :text min edit distance = 2
miss spelled word detected (texr) the suggested corrections sorted by MED are :tear min edit distance = 2
miss spelled word detected (edito) the suggested corrections is : ['edith', 'edit', 'editor']
miss spelled word detected (edito) the suggested corrections sorted by MED are :edit min edit distance = 1
miss spelled word detected (edito) the suggested corrections sorted by MED are :editor min edit distance = 1
miss spelled word detected (edito) the suggested corrections sorted by MED are :edith min edit distance = 2


In [20]:
q2_solution_MED('dhis is a mass speld wrd',vocab)

miss spelled word detected (dhis) the suggested corrections is : ['this', 'dais', 'phis', 'his']
miss spelled word detected (dhis) the suggested corrections sorted by MED are :his min edit distance = 1
miss spelled word detected (dhis) the suggested corrections sorted by MED are :this min edit distance = 2
miss spelled word detected (dhis) the suggested corrections sorted by MED are :dais min edit distance = 2
miss spelled word detected (dhis) the suggested corrections sorted by MED are :phis min edit distance = 2
miss spelled word detected (speld) the suggested corrections is : ['spend', 'spell', 'speed', 'sped']
miss spelled word detected (speld) the suggested corrections sorted by MED are :sped min edit distance = 1
miss spelled word detected (speld) the suggested corrections sorted by MED are :spend min edit distance = 2
miss spelled word detected (speld) the suggested corrections sorted by MED are :spell min edit distance = 2
miss spelled word detected (speld) the suggested correc

##### Language Model

In [21]:
def generate_ngrams(text, n):
    # Convert the text to lowercase and remove punctuation
    text = text.lower()
    text = text.replace(".", "").replace(",", "").replace("!", "").replace("?", "")
    
    # Tokenize the text into individual words
    words = text.split()
    
    # Generate n-grams
    n_grams = []
    for i in range(len(words) - n + 1):
        n_gram = (words[i:i+n])
        n_grams.append(tuple(n_gram))
    
    return n_grams

# Example usage
sentence = "This is an example sentence."
result = generate_ngrams(sentence,2)
print(result)


[('this', 'is'), ('is', 'an'), ('an', 'example'), ('example', 'sentence')]


In [22]:
from nltk.corpus import brown

In [23]:
sentences = brown.sents()

In [24]:
def count_n_grams(data, n, start_token='<s>', end_token = '<e>'):
    """
    Count all n-grams in the data
    
    Args:
        data: List of lists of words
        n: number of words in a sequence
    
    Returns:
        A dictionary that maps a tuple of n-words to its frequency
    """
    
    # Initialize dictionary of n-grams and their counts
    n_grams = {}

    ### START CODE HERE (Replace instances of 'None' with your code) ###
    
    # Go through each sentence in the data
    for sentence in data: # complete this line
        
        # prepend start token n times, and  append <e> one time
        sentence = [s.lower() for s in sentence]
        sentence = ([start_token] * n)+ sentence + [end_token]
        
        # convert list to tuple
        # So that the sequence of words can be used as
        # a key in the dictionary
        sentence = tuple(sentence)
        
        # Use 'i' to indicate the start of the n-gram
        # from index 0
        # to the last index where the end of the n-gram
        # is within the sentence.
        m = len(sentence) if n==1 else len(sentence)-1
        for i in range(m): # complete this line

            # Get the n-gram from i to i+n
            n_gram = sentence[i:i+n]

            # check if the n-gram is in the dictionary
            if n_gram in n_grams.keys(): # complete this line
            
                # Increment the count for this n-gram
                n_grams[n_gram] += 1
            else:
                # Initialize this n-gram count to 1
                n_grams[n_gram] = 1
    
            ### END CODE HERE ###
    return n_grams

In [25]:
bi_grams = count_n_grams(sentences,2)
uni_grams = count_n_grams(sentences,1)

In [27]:
def estimate_probability(word, previous_n_gram, uni_grams, bi_grams, vocabulary_size, k=1.0):
    """
    Estimate the probabilities of a next word using the n-gram counts with k-smoothing
    
    Args:
        word: next word
        previous_n_gram: A sequence of words of length n
        n_gram_counts: Dictionary of counts of n-grams
        n_plus1_gram_counts: Dictionary of counts of (n+1)-grams
        vocabulary_size: number of words in the vocabulary
        k: positive constant, smoothing parameter
    
    Returns:
        A probability
    """
    denominator = k*vocabulary_size
    numerator = k
    if (word,) in uni_grams:
      denominator += uni_grams[(word,)]
    if(previous_n_gram,word) in bi_grams:
      numerator += bi_grams[(previous_n_gram,word)]

    probability = numerator/denominator
    # ### END CODE HERE ###
    
    return np.log(probability)


In [28]:
len(list(set(vocab)))

49815

In [29]:
len(uni_grams)

49817

In [30]:
estimate_probability('am','i',uni_grams,bi_grams,len(list(set(vocab))))

-5.483279664283628

In [31]:
estimate_probability('hard','very',uni_grams,bi_grams,len(list(set(vocab))))

-9.721505937955271

In [32]:
estimate_probability('afa','rr',uni_grams,bi_grams,len(list(set(vocab))))

-10.816071422478958

In [33]:
estimate_probability('granted','for',uni_grams,bi_grams,len(list(set(vocab))))

-8.419299677676308

In [34]:
1/len(list(set(vocab)))

2.007427481682224e-05

In [35]:
def q2_solution_PorbLangModel(text,vocab):
  words = re.findall(r'\w+', text.lower())
  corrected_text=''
  for word in words:
    if word not in vocab :
      corrected_word = get_valid_corrections(word, vocab)
      if len(corrected_word) == 1:
        print("miss spelled word detected ("+word+") the suggested corrections is :"+ str(corrected_word))
      else:
        print("miss spelled word detected ("+word+") the suggested corrections are :"+ str(corrected_word))
        sorted_corrections=sorted(corrected_word,key=lambda x:estimate_probability(x,word,uni_grams, bi_grams, len(vocab)),reverse=True)
        for sor_corr in sorted_corrections:
          print("miss spelled word detected ("+word+") the suggested corrections sorted by probability is :"+ str(sor_corr)+" probability = "+str(estimate_probability(sor_corr,word,uni_grams, bi_grams, len(vocab))))

In [36]:
q2_solution_PorbLangModel('I enjoy writinf texr edito',vocab)

miss spelled word detected (writinf) the suggested corrections is :['writing']
miss spelled word detected (texr) the suggested corrections are :['tex', 'text', 'tear']
miss spelled word detected (texr) the suggested corrections sorted by probability is :tex probability = -13.9649584828678
miss spelled word detected (texr) the suggested corrections sorted by probability is :tear probability = -13.96496709466355
miss spelled word detected (texr) the suggested corrections sorted by probability is :text probability = -13.965009291390725
miss spelled word detected (edito) the suggested corrections are :['edith', 'edit', 'editor']
miss spelled word detected (edito) the suggested corrections sorted by probability is :edit probability = -13.964959344050714
miss spelled word detected (edito) the suggested corrections sorted by probability is :edith probability = -13.964961066414313
miss spelled word detected (edito) the suggested corrections sorted by probability is :editor probability = -13.96

In [37]:
q2_solution_PorbLangModel('it is a bad habbitr to take thinds for grantef',vocab)

miss spelled word detected (habbitr) the suggested corrections are :['habits', 'rabbits', 'babbitt', 'rabbit', 'habit']
miss spelled word detected (habbitr) the suggested corrections sorted by probability is :babbitt probability = -13.964959344050714
miss spelled word detected (habbitr) the suggested corrections sorted by probability is :rabbits probability = -13.964961927594999
miss spelled word detected (habbitr) the suggested corrections sorted by probability is :rabbit probability = -13.96496709466355
miss spelled word detected (habbitr) the suggested corrections sorted by probability is :habits probability = -13.964975706385136
miss spelled word detected (habbitr) the suggested corrections sorted by probability is :habit probability = -13.964977428720553
miss spelled word detected (thinds) the suggested corrections are :['thirds', 'things', 'thinks']
miss spelled word detected (thinds) the suggested corrections sorted by probability is :thirds probability = -13.964961066414313
mis